In [34]:
def parse_orders(IS_labels, NER_labels, tokens):
    result = {"ORDER": {"PIZZAORDER": [], "DRINKORDER": []}}
    pizza_order = {}
    drink_order = {}
    
    current_section = None
    current_number = None
    toppings = []
    style = []
    drink_type = None
    
    # Helper to append toppings
    def append_topping(topping_list, not_topping, topping_name):
        topping_list.append({
            "NOT": not_topping,
            "Quantity": None,  # Default null as not provided
            "Topping": topping_name
        })
    
    def append_style(style_list, not_style, style_name):
        style_list.append({
            "NOT": not_style,
            "Style": style_name
        })

    # Iterate through the IS and NER labels
    for i, (is_label, ner_label) in enumerate(zip(IS_labels, NER_labels)):
        token = tokens[i]  # Assuming you have tokens to map

        # Identify PIZZAORDER Section
        if is_label == "B-PIZZAORDER":
            if pizza_order:  # Add previous pizza order
                pizza_order["AllTopping"] = toppings
                result["ORDER"]["PIZZAORDER"].append(pizza_order)
            # Reset for the new pizza order
            pizza_order = {}
            toppings = []
            style = []
            current_section = "PIZZAORDER"
            current_number = None  # Reset number

        # Identify DRINKORDER Section
        elif is_label == "B-DRINKORDER":
            if drink_order:  # Add previous drink order
                result["ORDER"]["DRINKORDER"].append(drink_order)
            # Reset for the new drink order
            drink_order = {}
            current_section = "DRINKORDER"
            current_number = None  # Reset number

        # Extract NUMBER and assign based on context
        if ner_label == "B-NUMBER":
            current_number = token

        # Handle PIZZAORDER Details
        if current_section == "PIZZAORDER":
            if ner_label == "B-SIZE":
                pizza_order["SIZE"] = token
            pizza_order["NUMBER"] = current_number  # Update pizza quantity
            
            if ner_label == "B-TOPPING":
                append_topping(toppings, False, token)
            elif ner_label == "I-TOPPING" and toppings:
                toppings[-1]["Topping"] += f" {token}"  

            if ner_label == "B-STYLE":
                append_style(style, False, token)
                pizza_order["STYLE"] = style
            elif ner_label == "I-STYLE" and style:
                style[-1]["TYPE"] += f" {token}"  
            
            if ner_label == "B-COMPLEX_QUANTITY":
                append_topping(toppings, False, token)
                toppings[-1]["Quantity"] = token  # Complex topping quantity (e.g., "extra")
            elif ner_label == "I-COMPLEX_TOPPING" and toppings:
                toppings[-1]["Topping"] = token
                
            if ner_label == "B-NOT_TOPPING":
                append_topping(toppings, True, token)
            elif ner_label == "I-NOT_TOPPING" and toppings:
                toppings[-1]["Topping"] += f" {token}"
                
            if ner_label == "B-NOT_STYLE":
                append_style(style, True, token)
                pizza_order["STYLE"] = style
            elif ner_label == "I-NOT_STYLE" and style:
                style[-1]["TYPE"] += f" {token}"
                

            

        # Handle DRINKORDER Details
        elif current_section == "DRINKORDER":
            if ner_label == "B-DRINKTYPE":
                drink_type = token
                drink_order = {"NUMBER": current_number, "DRINKTYPE": drink_type}
            elif ner_label == "I-DRINKTYPE" and drink_order:
                drink_order["DRINKTYPE"] += f" {token}"  # Multi-word drink type

    # Append any pending orders
    if pizza_order:
        pizza_order["AllTopping"] = toppings        
        result["ORDER"]["PIZZAORDER"].append(pizza_order)
    if drink_order:
        result["ORDER"]["DRINKORDER"].append(drink_order)

    return result


# Input Data
NER_labels = ['O', 'O', 'O', 'O', 'B-NUMBER', 'B-SIZE',  'O','O','B-NOT_STYLE', 'O', 'B-TOPPING', 'O', 'B-TOPPING', 'I-TOPPING', 'O', 
              'B-NUMBER', 'B-SIZE', 'O', 'O', 'B-TOPPING', 'O', 'B-COMPLEX_QUANTITY', 'I-COMPLEX_TOPPING', 
              'O', 'B-NUMBER', 'B-SIZE', 'O', 'O', 'B-TOPPING', 'O', 'B-TOPPING']

IS_labels = ['O', 'O', 'O', 'O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER','I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 
             'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 
             'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'O', 'B-PIZZAORDER', 
             'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER']

source = "i want to order two medium pizzas not chicago with sausage and black olives and two medium pizzas with pepperoni and extra cheese and three large pizzas with pepperoni and sausage"
# IS_labels = ['O', 'O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'O', 
#              'B-DRINKORDER', 'I-DRINKORDER', 'O', 'B-DRINKORDER', 'I-DRINKORDER', 'O', 'B-DRINKORDER', 
#              'I-DRINKORDER']

# NER_labels = ['O', 'O', 'B-NUMBER', 'O', 'O', 'B-NOT_TOPPING', 'I-NOT_TOPPING', 'O', 
#               'B-NUMBER', 'B-DRINKTYPE', 'O', 'B-NUMBER', 'B-DRINKTYPE', 'O', 
#               'B-NUMBER', 'B-DRINKTYPE']

# source = "i'd like three pizzas no american cheese and a sprite and five fantas and one perrier"

tokens = source.split()
print(len(tokens), len(IS_labels), len(NER_labels))

# Run the function
output = parse_orders(IS_labels, NER_labels, tokens)

# Write the output in a JSON file
import json
with open("output.json", "w") as f:
    json.dump(output, f, indent=4)

# Print the output
print(json.dumps(output, indent=4))


31 31 31
{
    "ORDER": {
        "PIZZAORDER": [
            {
                "NUMBER": "two",
                "SIZE": "medium",
                "STYLE": [
                    {
                        "NOT": true,
                        "Style": "chicago"
                    }
                ],
                "AllTopping": [
                    {
                        "NOT": false,
                        "Quantity": null,
                        "Topping": "sausage"
                    },
                    {
                        "NOT": false,
                        "Quantity": null,
                        "Topping": "black olives"
                    }
                ]
            },
            {
                "NUMBER": "two",
                "SIZE": "medium",
                "AllTopping": [
                    {
                        "NOT": false,
                        "Quantity": null,
                        "Topping": "pepperoni"
                    },
     